# SimpleITK

The following short demo for this session was prepared using examples provided at SimpleITK Notebooks:

http://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/

Read more about SimpleITK (and ITK in general):

http://www.simpleitk.org/



## Setting the enviroment for simpleITK

At this session, we will use only a few 2D images to explain basic concepts in image registration, but if you would like to run the additional jupyter notebooks (for more advanced topics in image regitration or image segmentation), it is strongly advised to download all of the data before you run the notebook.

In case of any problems with setting enviroment, see more:
http://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/Python_html/00_Setup.html

In [5]:
import importlib

# check that all packages are installed (see requirements.txt file)
required_packages = {'jupyter', 
                     'numpy',
                     'matplotlib',
                     'ipywidgets',
                     'scipy',
                     'pandas',
                     'SimpleITK'
                    }

for package in required_packages:
    try:
        p = importlib.import_module(package)
        print(package, 'is installed')
    except ImportError:
        print(package, 'is missing')

numpy is installed
matplotlib is installed
ipywidgets is installed
pandas is installed
scipy is installed
SimpleITK is installed
jupyter is installed


If you need to install SimpleITK

__pip install SimpleITK__

If any issues, see more:
https://simpleitk.readthedocs.io/en/master/Documentation/docs/source/installation.html

In [7]:
pip install SimpleITK

Note: you may need to restart the kernel to use updated packages.


## Check SimpleITK version and download data:


In [3]:
import SimpleITK as sitk

print(sitk.Version())


SimpleITK Version: 1.2.0 (ITK 4.13)
Compiled: Jan  4 2019 04:43:21



In [4]:
import sys, os
download_script_location = os.path.abspath(os.path.join('..','Utilities'))
if not download_script_location in sys.path:
    sys.path.append(download_script_location)
    
from downloaddata import fetch_data, fetch_data_all



fetch_data_all(os.path.join('..','Data'), os.path.join('..','Data','manifest.json'))

Fetching cthead1.png
Fetching VM1111Shrink-RGB.png
Fetching 2th_cthead1.png
Fetching nac-hncma-atlas2013-Slicer4Version/Data/A1_grayT1.nrrd
Fetching nac-hncma-atlas2013-Slicer4Version/Data/A1_grayT2.nrrd
Fetching nac-hncma-atlas2013-Slicer4Version/Data/hncma-atlas.nrrd
Fetching B1.tiff
Fetching B2.tiff
Fetching B1_fixed_B2_moving0GenericAffine.mat
Fetching B1_fixed_B2_movingComposite.h5
Fetching coins.png
Fetching a_vm1108.png
Fetching vm_head_rgb.mha
Fetching vm_head_mri.mha
Fetching training_001_ct.mha
Fetching training_001_mr_T1.mha
Fetching ct_T1.standard
Fetching CIRS057A_MR_CT_DICOM/readme.txt
Fetching SimpleITK.jpg
Fetching spherical_fiducials.mha
Fetching POPI/meta/00-P.mhd
Fetching POPI/meta/10-P.mhd
Fetching POPI/meta/20-P.mhd
Fetching POPI/meta/30-P.mhd
Fetching POPI/meta/40-P.mhd
Fetching POPI/meta/50-P.mhd
Fetching POPI/meta/60-P.mhd
Fetching POPI/meta/70-P.mhd
Fetching POPI/meta/80-P.mhd
Fetching POPI/meta/90-P.mhd
Fetching POPI/landmarks/00-Landmarks.pts
Fetching POPI/la